In [1]:
import pickle
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']
desired_length = 84
def preprocess_data(data_aux, desired_length=84):
    if len(data_aux) < desired_length:
        # Pad with zeros if data_aux is shorter than the desired length
        data_aux = np.pad(data_aux, (0, desired_length - len(data_aux)), 'constant')
    elif len(data_aux) > desired_length:
        # Truncate if data_aux is longer than the desired length
        data_aux = data_aux[:desired_length]
    return data_aux


In [ ]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(model_complexity=1,static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Define the desired length for padding/truncation
desired_length = 84

def preprocess_data(data_aux, desired_length=84):
    if len(data_aux) < desired_length:
        data_aux = np.pad(data_aux, (0, desired_length - len(data_aux)), 'constant')
    elif len(data_aux) > desired_length:
        data_aux = data_aux[:desired_length]
    return data_aux

cap = cv2.VideoCapture(0)

while True:
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()
    if not ret:
        break  # Exit loop if no frame is captured

    H, W, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10
        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10
        
        data_aux_padded = preprocess_data(data_aux, desired_length)
        prediction = model.predict([np.asarray(data_aux_padded)])
        # print(prediction)
        predicted_character = str(prediction[0])

        # Uncomment the following lines if you want to display the predictions
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                    cv2.LINE_AA)
        
        

    cv2.imshow('frame', frame)

    # Check for key press to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [13]:
import pickle
import cv2
import mediapipe as mp
import numpy as np
import time

# Load the trained model
model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']

# Desired length for padding/truncation
desired_length = 84

def preprocess_data(data_aux, desired_length=84):
    if len(data_aux) < desired_length:
        # Pad with zeros if data_aux is shorter than the desired length
        data_aux = np.pad(data_aux, (0, desired_length - len(data_aux)), 'constant')
    elif len(data_aux) > desired_length:
        # Truncate if data_aux is longer than the desired length
        data_aux = data_aux[:desired_length]
    return data_aux

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(model_complexity=1, static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Initialize variables for predicted text
letter_buffer = []
display_text = ""
last_update_time = 0
add_letter_interval = 2  # Interval to add a new letter (seconds)

cap = cv2.VideoCapture(0)

while True:
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()
    if not ret:
        break  # Exit loop if no frame is captured

    H, W, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10
        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10
        
        data_aux_padded = preprocess_data(data_aux, desired_length)
        prediction = model.predict([np.asarray(data_aux_padded)])  # Predict using the loaded model
        predicted_character = str(prediction[0])  # Extract predicted character
        
        # Add letter only after the specified interval
        current_time = time.time()
        if current_time - last_update_time > add_letter_interval:
            if predicted_character.isalpha():  # Only consider alphabetic predictions
                letter_buffer.append(predicted_character.upper())  # Convert to uppercase
                # Maintain rolling 8 characters
                if len(letter_buffer) > 8:
                    letter_buffer.pop(0)  # Remove the oldest character
                last_update_time = current_time  # Reset the timer

        # Combine buffer into a spaced word
        display_text = " ".join(letter_buffer)

        # Draw the bounding box and predicted character
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3, cv2.LINE_AA)

    # Display the accumulated word at the top of the screen
    cv2.rectangle(frame, (0, 0), (640, 40), (245, 117, 16), -1)  # Orange rectangle
    cv2.putText(frame, display_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 140, 255), 2, cv2.LINE_AA)  # Orange text

    cv2.imshow('frame', frame)

    # Check for key press to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
